In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# Streaming = we can check response of model
# callbacks = allow you to see responses as it is being generated
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients.",
        ),
        ("human", "I want to cook {cuisine} food."),
    ],
)

chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it.",
        ),
        ("human", "{recipe}"),
    ]
)

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe" : chef_chain} | veg_chef_chain

final_chain.invoke({
    "cuisine" : "indian"
})

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    # model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

# FewShotPromptTemplate : 템플릿을 직접 만드는 것 보다, 예시를 들어 보여주는 것이 더 쉬움
# 왜냐하면 모델이 텍스트를 만들어낼 것이기 때문 - 이것을 통해 예제를 형식화 할 수 있음
# 그리고 이 방식이 LLM을 만드는 방식이고, 더 나은 LLM을 만들 수 있음

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)
# this is for formating examples
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers"),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

# prompt.format(country= "Germany")
chain = final_prompt | chat

chain.invoke({"country": "Japan"})


        I know this:
        Capital: Tokyo
        Language: Japanese
        Food: Sushi and Ramen
        Currency: Japanese Yen
        

AIMessageChunk(content='\n        I know this:\n        Capital: Tokyo\n        Language: Japanese\n        Food: Sushi and Ramen\n        Currency: Japanese Yen\n        ')